In [5]:
import cv2   #open cv library
import numpy as np # array in python
import time

In [6]:


# web camera
cap = cv2.VideoCapture('video.mp4')

#initiaize
algo = cv2.bgsegm.createBackgroundSubtractorMOG()

count_line_pos = 550
width = 80
height = 80
detect = []
offset = 6

#vehicle counter
counter = 0

#functions
def center_handle(x,y,w,h):
    x1 = int(w/2)
    y1 = int(h/2)
    cx = x+x1
    cy = y+y1
    return cx,cy


while True: #infinite loop
    
    #reads frames
    ret,frame1 = cap.read()
    
    #converts to gray scale
    grey = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    
    #blurs
    blur = cv2.GaussianBlur(grey,(3,3),5)
    
    
    
    #applys all the effects above
    img_sub = algo.apply(blur)
    
    #dilates the image
    dilat = cv2.dilate(img_sub,np.ones((5,5)))
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    dilatada = cv2.morphologyEx(dilat,cv2.MORPH_CLOSE, kernel)
    dilatada = cv2.morphologyEx(dilatada,cv2.MORPH_CLOSE, kernel)
    
    counterShape,h = cv2.findContours(dilatada,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    #draws red line
    cv2.line(frame1,(25,count_line_pos),(1200,count_line_pos),(0,0,255),3)  
    
    #draws rectangle
    for (i,c) in enumerate(counterShape):
        (x,y,w,h) = cv2.boundingRect(c)
        validate_counter = (w >= width) and (h>=height)
        if not validate_counter:
            continue
        
        #draws bounding box around vehicle
        cv2.rectangle(frame1,(x,y),(x+w,y+h),(0,255,0),2)
        
        #displays vehicle number
        cv2.putText(frame1,'Vehicle'+str(counter),(x,y-20),cv2.FONT_HERSHEY_SIMPLEX,1,(255,244,0),3)
        
        
        #points on vehicle
        center = center_handle(x,y,w,h)
        detect.append(center)
        cv2.circle(frame1,center,4,(0,0,255),-1)
        
        # increments counter
        for (x,y) in detect:
            if y<(count_line_pos+offset) and y>(count_line_pos-offset):
                counter+=1
                
                cv2.line(frame1,(25,count_line_pos),(1200,count_line_pos),(0,127,255),3)
                detect.remove((x,y))
    
    #display vehicle counter
    cv2.putText(frame1,"VEHICLE COUNTER : " + str(counter),(450,70),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),5)
            
    
    #displays the video
    cv2.imshow('Video Original',frame1)
    
    if cv2.waitKey(1) == 13:   # enter key breaks the loop
        break
        
    time.sleep(0.01)
        
cv2.destroyAllWindows()
cap.release()